In [4]:
import sqlite3
import numpy
dbName = 'movies2.db'
inputFile = 'ratings.csv'
con = sqlite3.connect(dbName)
cur = con.cursor()


class myCorr:
    def __init__(self):
        self.count = 0
        self.list1 = []
        self.list2 = []
    def step(self, value1,value2):
        self.list1.append(value1)
        self.list2.append(value2)
    def finalize(self):
        #print(self.list1[:10])
        #print(self.list2[:10])
        if len(self.list1) and len(self.list2):
            self.correlation = numpy.corrcoef(self.list1,self.list2)[0][1]
        else:
            self.correlation = 0.0
        return self.correlation

con.create_aggregate("corr", 2, myCorr)


In [2]:
import convert

#convert.convertJSON(dbName,inputFile,("reviewerID","overall","asin")) #these keys may change based on convert.py
convert.convertCSV(dbName,inputFile,[0,2,1]) #has to be user,rating,itemId for now
drop = "drop table if exists itemInfo"

create = """
create table if not exists itemInfo (
itemID varchar(25),
occurances int
)
"""
createIndex = "create index idx_1 on itemRatings(itemID,user,rating)"
createID = """
insert into itemInfo(itemID) select distinct itemID from itemRatings
"""
occurances = """
UPDATE itemInfo
SET occurances = (select count(*) t from itemRatings where itemID = itemInfo.itemID)
where exists(select count(*) t from itemRatings where itemID = itemInfo.itemID)
"""
cur.execute(drop)
print("dropped")
cur.execute(create)
print("created")
cur.execute(createIndex)
print("indexed")
cur.execute(createID)
print("foreign keyed")
cur.execute(occurances)
print("frequency counted")
con.commit()

# cur.execute("insert into petSupplies (rating) select rating - (select average from (select AVG(rating) as average,username as usern from petSupplies group by username) where usern = petSupplies.username) from petSupplies ") 
# con.commit()

['userId', 'movieId', 'rating', 'timestamp']
dropped
created
indexed
foreign keyed
frequency counted


In [ ]:

from datetime import datetime

def calcCorrelations(item,threshold,frequency=.05):
    small = f"""select itemID1,correlation,occurances from
    (select corr(rating1,rating2) correlation,itemID2,itemID1 from 
    ((select itemID as itemID2,user as user2,rating as rating2  from petSupplies where itemID = "{item}") 
    inner join  (select itemID itemID1,user as user1,rating as rating1 from petSupplies)
    on  user1 = user2) group by itemID1)  
    inner join (select itemID id, occurances from itemInfo where occurances > {threshold}) on itemID1=id order by correlation desc"""
    noOccurances = f"""select itemID1,correlation,occurances from
    (select corr(rating1,rating2) correlation,itemID2,itemID1 from 
    ((select itemID as itemID2,user as user2,rating as rating2  from petSupplies where itemID = "{item}") 
    inner join  (select itemID itemID1,user as user1,rating as rating1 from petSupplies)
    on  user1 = user2) group by itemID1) order by correlation desc"""
    start = datetime.now()
    cur.execute(noOccurances)
    result = cur.fetchmany(100)
    print((datetime.now()-start))
    return result
print(calcCorrelations("B002CJIPEK",10))

OperationalError: no such table: petSupplies

In [8]:
import csv
f = open("movies.csv")
reader = csv.reader(f,decoding='utf8')
next(reader)
for row in reader:
    #print(row)
    try:
        con.execute(f"""Update itemInfo 
        SET name = ?
        where itemID = ?
        """,(row[1],row[0]))
    except:
        continue

TypeError: 'decoding' is an invalid keyword argument for this function